![logo](https://github.com/jeanmatheuss/SB-2023-python/blob/main/imgs/logo.jpg?raw=true)

---
## **Projeto 1** | Explorando IA Generativa em Pipeline de ETL com Python
[Jean Matheus](https://www.linkedin.com/in/jean-matheus-sm/)

---


# Personalização de Mensagens de Marketing com IA Generativa

## Descrição
Este projeto foi desenvolvido como parte do Santander Bootcamp 2023 - Ciência de Dados com Python, com o obejtivo de envovler os clientes de maneira mais personalizada usando o poder da Inteligência Artificial Generativa.

## Contexto
Como cientista de dados no Santander, nossa tarefa era melhorar o envolvimento dos clientes por meio de mensagens de marketing altamente personalizadas.    
O objetivo é utilizar  a Inteligência Artificial Generativa para criar mensagens que ressoem com cada cliente individualmente.

Aqui estão as estapas que seguiremos:
1. Extração de Dados
2. Obtenção de Dados  dos Clientes
3. Transformação de Dados
4. Geração de Mensagens Personalizadas
5. Atualização de Dados dos Clientes

### Pacotes e Bibliotecas

In [1]:
import json
import pandas as pd
import seaborn as sns
import numpy as np
import requests

In [ ]:
# Criando DF a partir do arquivo SDW2023-1 baixado diretamente da API

df = pd.read_json('SDW2023-1.json')

In [2]:
# URL da API para a consulta dos clientes

url = 'https://sdw-2023-prd.up.railway.app'

In [65]:
# Criando DF diretamente da API

response = requests.get(f'{url}/users')
clients = response.json() if response.status_code == 200 else None


## Não necessários no momento

In [67]:
clients_df = pd.DataFrame(clients)
clients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 971 entries, 0 to 970
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        971 non-null    int64 
 1   name      969 non-null    object
 2   account   969 non-null    object
 3   card      969 non-null    object
 4   features  971 non-null    object
 5   news      971 non-null    object
dtypes: int64(1), object(5)
memory usage: 45.6+ KB


> Como usaremos esse projeto apenas para demonstração, usar o *clientes_df* ficaria inviável pois são quase 1400 clientes fictícios e como a API do ChatGPT que irá gerar a mensagem não é gratuíta, vamos usar apenas os 5 prmeiros clientes.

In [12]:
clients_df = clients_df.sort_values(by='id', ascending=True)

In [5]:
clients_df = clients_df.head()
clients_df

,id,name,account,card,features,news
0,3210,None,None,None,[],"[{'id': 6736, 'icon': 'string', 'description':..."
1,3334,Marcos,"{'id': 3538, 'number': '00003-1', 'agency': '0...","{'id': 3228, 'number': '**** **** **** 1236', ...","[{'id': 1030, 'icon': 'string', 'description':...","[{'id': 6773, 'icon': 'string', 'description':..."
2,3335,João,"{'id': 3539, 'number': '00004-1', 'agency': '0...","{'id': 3229, 'number': '**** **** **** 1237', ...","[{'id': 1031, 'icon': 'string', 'description':...","[{'id': 6774, 'icon': 'string', 'description':..."
3,1,Devweekerson,"{'id': 1, 'number': '01.097954-4', 'agency': '...","{'id': 1, 'number': 'xxxx xxxx xxxx 1111', 'li...","[{'id': 1, 'icon': 'https://digitalinnovationo...","[{'id': 1, 'icon': 'https://digitalinnovationo..."
4,3368,Alfonso,"{'id': 3578, 'number': '12001-1', 'agency': '0...","{'id': 3264, 'number': '**** **** **** 1851', ...",[],[]


In [15]:
lista_id = clients_df['id'].to_list()
lista_id

[1, 13, 14, 15, 17]

In [14]:
# Flatten da coluna account
clients_account = pd.json_normalize(clients_df['account'])
clients_account

,id,number,agency,balance,limit
0,1,01.097954-4,2030,624.12,1000.0
1,20,00010-8,2003,500.00,1000.0
2,21,00110-2,2003,500.00,1000.0
3,22,0001-1,0001,0.00,500.0
4,905,**** **** **45,Banku,0.00,0.0


In [15]:
# Flatten da coluna card
clients_card = pd.json_normalize(clients_df['card'])
clients_card

,id,number,limit
0,1,xxxx xxxx xxxx 1111,2000.0
1,13,**** **** **** 0112,500.0
2,14,**** **** **** 0322,500.0
3,15,1234 1234 1234 1111,1000.0
4,836,**** **** **67,0.0


In [35]:
teste = clients[0]
teste['name']

'leonardo fogaca'

## Voltando

In [6]:
# API ChatGpt
import os
from getpass import getpass

api_key = getpass()


In [63]:
import openai
import time

openai.api_key = api_key

def gerenate_ai_news(user, limit):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Você é um especialista em marketing e em marketing bancário."
            },
            {
                "role": "user",
                "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos, que possuí um limite de:{limit['limit']} (máximo 140 caracteres, sem o uso de #)"
            }
        ]
    )
    return completion.choices[0].message.content.strip('\"')

# escolhendo os 3 primeiros clientes do banco de dados da API
for user in clients[1:4]:
    limit = user['account']
    news = gerenate_ai_news(user, limit)
    print(news)
    user['news'].append({
        "icon": "https://raw.githubusercontent.com/digitalinnovationone/santander-dev-week-2023-api/665804c8231b32923b03b30ab832ccee33cf2cc3/docs/icons/credit.svg",
        "description": news
    })


Invista no seu futuro financeiro! Com apenas R$2000.0, você pode potencializar seus ganhos e conquistar seus objetivos. Não perca essa oportunidade!
João, invista agora para garantir seu futuro financeiro. Aproveite o limite de R$2200.0 disponível!
Invista no seu futuro com sabedoria! Com um limite de 1000.0, você pode alcançar grandes resultados financeiros. Seja estratégico e comece agora.


In [76]:
# Fazendo o upload para API com a lista de 'news' atualizada, com a nova mensagem gerada pelo ChatGPT

def uptade_user(user):
    response = requests.put(f"{url}/users/{user['id']}", json=user)

    return print(f"User {user['name']} updated: True!") if response.status_code == 200  else print(f"User {user['name']} updated: False! Error: {response.status_code}")

for user in clients[1:4]:
    success = uptade_user(user)


User Marcos updated: True!
User João updated: True!
User Devweekerson updated: False! Error: 422


In [59]:
clients[1]

{'id': 3334,
 'name': 'Marcos',
 'account': {'id': 3538,
  'number': '00003-1',
  'agency': '0001',
  'balance': 0.0,
  'limit': 2000.0},
 'card': {'id': 3228, 'number': '**** **** **** 1236', 'limit': 2000.0},
 'features': [{'id': 1030, 'icon': 'string', 'description': 'string'}],
 'news': [{'id': 6773, 'icon': 'string', 'description': 'string'},
  {'id': 7176,
   'icon': 'https://raw.githubusercontent.com/digitalinnovationone/santander-dev-week-2023-api/665804c8231b32923b03b30ab832ccee33cf2cc3/docs/icons/account.svg',
   'description': 'Maximize seu potencial financeiro e conquiste seus objetivos através dos investimentos. Seu futuro começa agora! 💰📈 #invista'},
  {'id': 7177,
   'icon': 'https://raw.githubusercontent.com/digitalinnovationone/santander-dev-week-2023-api/665804c8231b32923b03b30ab832ccee33cf2cc3/docs/icons/credit.svg',
   'description': 'Invista no seu futuro com segurança e rentabilidade: aproveite seu limite de R$2000 e faça seu dinheiro trabalhar por você. #Investim